In [ ]:
!pip install transformers datasets
!pip install --upgrade sympy
!pip install --upgrade datasets
!pip install evaluate
!pip install transformers
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 50.7 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Unins

In [ ]:
!pip install evaluate
import os
from datasets import Dataset
import torch
from PIL import Image
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
import torch
from torch import nn
import evaluate
import zipfile
import os
from torch.utils.data import DataLoader
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_path = "/content/drive/My Drive/data_validmask.zip"
output_dir = "/content/data"
os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"ZIP 파일이 해제되었습니다: {output_dir}")

ZIP 파일이 해제되었습니다: /content/data


In [ ]:
train_images_dir = "/content/data/content/car_damage_data_v/train/image"
train_masks_dir = "/content/data/content/car_damage_data_v/masks/train"

val_images_dir = "/content/data/content/car_damage_data_v/valid/image"
val_masks_dir = "/content/data/content/car_damage_data_v/masks/valid"

label_mapping = {'Background': 0, 'Damaged': 1}

id2label = {v: k for k, v in label_mapping.items()}
label2id = {k: v for k, v in label_mapping.items()}

print("id2label:", id2label)
print("label2id:", label2id)
print("Number of labels:", len(id2label))

id2label: {0: 'Background', 1: 'Damaged'}
label2id: {'Background': 0, 'Damaged': 1}
Number of labels: 2


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "nvidia/mit-b3"
processor = SegformerImageProcessor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)


config.json:   0%|          | 0.00/70.0k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/179M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b3 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)

In [ ]:
def load_data(images_dir, masks_dir):
    images = sorted(os.listdir(images_dir))
    masks = sorted(os.listdir(masks_dir))

    data = []
    for img_name, mask_name in zip(images, masks):
        image_path = os.path.join(images_dir, img_name)
        mask_path = os.path.join(masks_dir, mask_name)
        data.append({"image": image_path, "mask": mask_path})
    return data

In [ ]:
train_data = load_data(train_images_dir, train_masks_dir)
val_data = load_data(val_images_dir, val_masks_dir)

In [ ]:
def preprocess(example):
    image = Image.open(example["image"]).convert("RGB").resize((512, 512))
    mask = Image.open(example["mask"]).resize((512, 512))
    encoding = processor(image, mask, return_tensors="pt")
    encoding = {k: v.squeeze(0) for k, v in encoding.items()}
    return encoding

In [ ]:
train_dataset = Dataset.from_list(train_data).map(preprocess)
val_dataset = Dataset.from_list(val_data).map(preprocess)

train_dataset.set_format(type="torch", columns=["pixel_values", "labels"])
val_dataset.set_format(type="torch", columns=["pixel_values", "labels"])

Map:   0%|          | 0/7062 [00:00<?, ? examples/s]

Map:   0%|          | 0/1513 [00:00<?, ? examples/s]

In [ ]:
batch_size = 4

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

metric = evaluate.load("mean_iou")

In [ ]:
class IoULoss(nn.Module):
    def forward(self, logits, targets):
        smooth = 1.0
        probs = torch.softmax(logits, dim=1)[:, 1, :, :]
        resized_targets = nn.functional.interpolate(
            targets.unsqueeze(1).float(),
            size=probs.shape[-2:],
            mode="nearest",
        ).squeeze(1)
        intersection = (probs * resized_targets).sum()
        union = probs.sum() + resized_targets.sum() - intersection
        iou = (intersection + smooth) / (union + smooth)
        return 1 - iou

class CombinedLoss(nn.Module):
    def __init__(self, weight_iou=0.7, weight_ce=0.3):
        super().__init__()
        self.iou_loss = IoULoss()
        self.ce_loss = nn.CrossEntropyLoss()
        self.weight_iou = weight_iou
        self.weight_ce = weight_ce

    def forward(self, logits, targets):
        resized_targets = nn.functional.interpolate(
            targets.unsqueeze(1).float(),
            size=logits.shape[-2:],
            mode="nearest",
        ).squeeze(1).long()

        iou_loss = self.iou_loss(logits, resized_targets)
        ce_loss = self.ce_loss(logits, resized_targets)
        return self.weight_iou * iou_loss + self.weight_ce * ce_loss

In [ ]:
loss_fn = CombinedLoss(weight_iou=0.5, weight_ce=0.5)

In [ ]:
epochs = 3
lr = 5e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [ ]:
def evaluate_model(model, data_loader):
    model.eval()
    preds = []
    refs = []
    with torch.no_grad():
        for batch in data_loader:
            pixel_values = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(pixel_values=pixel_values)
            logits = outputs.logits

            upsampled_logits = nn.functional.interpolate(
                logits,
                size=labels.shape[-2:],
                mode="bilinear",
                align_corners=False,
            )
            pred_labels = upsampled_logits.argmax(dim=1).detach().cpu().numpy()
            refs.append(labels.detach().cpu().numpy())
            preds.append(pred_labels)
    preds = np.concatenate(preds, axis=0)
    refs = np.concatenate(refs, axis=0)
    results = metric.compute(
        predictions=preds,
        references=refs,
        num_labels=len(id2label),
        ignore_index=None,
        reduce_labels=processor.do_reduce_labels,
    )

    per_category_accuracy = results.pop("per_category_accuracy").tolist()
    per_category_iou = results.pop("per_category_iou").tolist()

    for i, label_name in id2label.items():
        results[f"accuracy_{label_name}"] = per_category_accuracy[i]
        results[f"iou_{label_name}"] = per_category_iou[i]

    return results

In [ ]:
best_mean_iou = -1.0
best_model_path = "best_model.pth"

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits
        loss = loss_fn(logits, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (step + 1) % 50 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{step+1}/{len(train_loader)}], Loss: {running_loss / (step+1):.4f}")

    val_results = evaluate_model(model, val_loader)
    print(f"Epoch [{epoch+1}/{epochs}] Validation Results: {val_results}")

    if val_results["mean_iou"] > best_mean_iou:
        best_mean_iou = val_results["mean_iou"]
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with mean_iou: {best_mean_iou:.4f}")

Epoch [1/3], Step [50/1766], Loss: 0.6246
Epoch [1/3], Step [100/1766], Loss: 0.5746
Epoch [1/3], Step [150/1766], Loss: 0.5462
Epoch [1/3], Step [200/1766], Loss: 0.5308
Epoch [1/3], Step [250/1766], Loss: 0.5225
Epoch [1/3], Step [300/1766], Loss: 0.5092
Epoch [1/3], Step [350/1766], Loss: 0.5015
Epoch [1/3], Step [400/1766], Loss: 0.4957
Epoch [1/3], Step [450/1766], Loss: 0.4934
Epoch [1/3], Step [500/1766], Loss: 0.4892
Epoch [1/3], Step [550/1766], Loss: 0.4866
Epoch [1/3], Step [600/1766], Loss: 0.4838
Epoch [1/3], Step [650/1766], Loss: 0.4806
Epoch [1/3], Step [700/1766], Loss: 0.4768
Epoch [1/3], Step [750/1766], Loss: 0.4741
Epoch [1/3], Step [800/1766], Loss: 0.4716
Epoch [1/3], Step [850/1766], Loss: 0.4705
Epoch [1/3], Step [900/1766], Loss: 0.4675
Epoch [1/3], Step [950/1766], Loss: 0.4656
Epoch [1/3], Step [1000/1766], Loss: 0.4640
Epoch [1/3], Step [1050/1766], Loss: 0.4618
Epoch [1/3], Step [1100/1766], Loss: 0.4589
Epoch [1/3], Step [1150/1766], Loss: 0.4573
Epoch [1

/usr/local/lib/python3.10/dist-packages/datasets/features/image.py:348: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


Epoch [1/3] Validation Results: {'mean_iou': 0.70163821976368, 'mean_accuracy': 0.7752154068731734, 'overall_accuracy': 0.944105497512767, 'accuracy_Background': 0.9781438496925058, 'iou_Background': 0.9412875038846312, 'accuracy_Damaged': 0.5722869640538409, 'iou_Damaged': 0.46198893564272875}
New best model saved with mean_iou: 0.7016
Epoch [2/3], Step [50/1766], Loss: 0.3652
Epoch [2/3], Step [100/1766], Loss: 0.3716
Epoch [2/3], Step [150/1766], Loss: 0.3687
Epoch [2/3], Step [200/1766], Loss: 0.3702
Epoch [2/3], Step [250/1766], Loss: 0.3714
Epoch [2/3], Step [300/1766], Loss: 0.3712
Epoch [2/3], Step [350/1766], Loss: 0.3723
Epoch [2/3], Step [400/1766], Loss: 0.3736
Epoch [2/3], Step [450/1766], Loss: 0.3756
Epoch [2/3], Step [500/1766], Loss: 0.3758
Epoch [2/3], Step [550/1766], Loss: 0.3758
Epoch [2/3], Step [600/1766], Loss: 0.3755
Epoch [2/3], Step [650/1766], Loss: 0.3740
Epoch [2/3], Step [700/1766], Loss: 0.3726
Epoch [2/3], Step [750/1766], Loss: 0.3739
Epoch [2/3], Step